1. Add fake 2014 data with gDP
1. Run a model for each product
1. Recursive model
1. Add Further Feature engineering - ambrosm https://www.kaggle.com/ambrosm/tpsjan22-03-linear-model/
1. SARIMAX model

* **Outliers**? - check for linear and Trees

* Catboost minmax scaling 

# Libraries

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

#import lightgbm as lgb 
#import xgboost as xgb
from catboost import CatBoostRegressor
#from sklearn.linear_model import LinearRegression,HuberRegressor,SGDRegressor
#from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

import optuna
import math

In [ ]:
LINEAR_DATE_AUG = False

OPTUNA = False
NUM_TRIALS = 400
SEED = 200

##Add DATA
ADD_2014 = False 

#Holidays
HOLIDAYS = True     
NEXT_HOLIDAY = True  

POST_PROCESSING = False
MODEL_TYPE = "Catboost"

VAL_SPLIT = "2017-12-31" #"2018-05-31"

In [ ]:
EPOCHS = 10000     
EARLY_STOPPING = 30

###########Come back to this   ---Doesnt work in CATBOOST (need to apply it)
SCALER_NAME = "MinMax"  #None MinMax  
SCALER = MinMaxScaler() 

obj is the objective function of the algorithm, i.e. what it's trying to maximize or minimize, e.g. "regression" means it's minimizing squared residuals.

Metric and eval are essentially the same. They are used for Early stopping 

# Load Data

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv",index_col = 0)
test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv",index_col = 0)
gdp_df = pd.read_csv('../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')
gdp_df.set_index('year', inplace=True)

if HOLIDAYS:
    holidays = pd.read_csv("../input/holidays-finland-norway-sweden-20152019/Holidays_Finland_Norway_Sweden_2015-2019.csv",usecols = ["Date","Country","Name"]                      )
    holidays.rename(columns = {"Date":"date","Country":"country","Name":"holiday"},inplace= True)
    holidays["holiday"]= 1
    holidays["holiday"]= holidays["holiday"].astype("int32")
    holidays["date"] = pd.to_datetime(holidays["date"])

In [ ]:
#Make date
train["date"] = pd.to_datetime(train["date"])
test["date"] = pd.to_datetime(test["date"])

In [ ]:
if ADD_2014:
    train_2014= train [train["date"]<"2016-01-01"]
    train_2014["date"] = train_2014["date"] - pd.DateOffset(years=1)
    train_2014["num_sold"] = train_2014["num_sold"]*0.98
    train = pd.concat([train_2014,train],axis=0,ignore_index=True)

In [ ]:
train.head()

# Functions 

In [ ]:
def public_hols(df):
    df = pd.merge(df, holidays, how='left', on=['date', 'country'])
    df.fillna(value = 0,inplace=True)
    
    return df

if HOLIDAYS:
    train = public_hols(train)
    test = public_hols(test)

In [ ]:
def get_gdp(row):
    country = 'GDP_' + row.country
    return gdp_df.loc[row.date.year, country]

In [ ]:
def engineer(df):
    #get GDP from file 
    df["gdp"] = df.apply(get_gdp, axis=1)   #improves Huber & Tweedie & catboost
    
    df["day"] = df["date"].dt.day
    df["dayofweek"] = df["date"].dt.dayofweek
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    
    #play around with if Tree model - each varies 
    #df['dayofyear'] = df['date'].dt.dayofyear                ### This can cause noise 
    df['inverse_dayofyear'] = 365 - df['date'].dt.dayofyear    # good for all  except catboost
    df.loc[df["year"] == 2016 , "inverse_dayofyear"] = df.loc[df["year"] == 2016 , "inverse_dayofyear"]+1     #Leap year in 2016
    df['quarter'] = 'Q' + df['date'].dt.quarter.astype(str)      # Good for lightgbm & Huber, bad for Tweedie & catboost
    #df['daysinmonth'] = df['date'].dt.days_in_month          ## Bad for all except Lightgbm

    #df["Friday"] = df["dayofweek"] ==4
    #df["Sat_sun"] = (df["dayofweek"] ==5) |(df["dayofweek"] ==6)
    
    if LINEAR_DATE_AUG:
        for country in ['Finland', 'Norway']:
            df[country] = df.country == country
        df['KaggleRama'] = df.store == 'KaggleRama'
        for product in ['Kaggle Mug', 'Kaggle Sticker']:
            df[product] = df['product'] == product

        df.drop(["country","store","product"], axis =1, inplace = True)

        # Seasonal variations (Fourier series)
        dayofyear = df.date.dt.dayofyear
        for k in range(1, 20):
            df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
            df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
            df[f'mug_sin{k}'] = df[f'sin{k}'] * df['Kaggle Mug']
            df[f'mug_cos{k}'] = df[f'cos{k}'] * df['Kaggle Mug']
            df[f'sticker_sin{k}'] = df[f'sin{k}'] * df['Kaggle Sticker']
            df[f'sticker_cos{k}'] = df[f'cos{k}'] * df['Kaggle Sticker']

    return df

In [ ]:
train = engineer(train)
test = engineer(test)

#Sometimes this helps sometimes is doesnt - catboost encodes within model
categorical_feats = [
    "country","store","product",
                     "quarter", 
                     #"weekend_ind"
                    ]

In [ ]:
def next_holiday(x):
    i=1
    while sum(holidays["date"] == pd.Timestamp(x) + pd.DateOffset(days=i)) ==0:
        i+=1
        if i >200:
            i=0
            break
            break
    return i

if NEXT_HOLIDAY:
    holidays["date"] = pd.to_datetime(holidays["date"])
    train["to_holiday"] = train["date"].apply(lambda x : next_holiday(x))
    test["to_holiday"] = test["date"].apply(lambda x : next_holiday(x))

In [ ]:
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

In [ ]:
def scale_data(X_train, X_test, test):
    scaler= SCALER
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    test = scaler.transform(test)
    
    return X_train, X_test, test

In [ ]:
train.head()

### One HotEncoder

In [ ]:
# Catboost does encoding in training 
train = pd.get_dummies(train,columns= categorical_feats)
test = pd.get_dummies(test,columns= categorical_feats)

In [ ]:
train.head()

In [ ]:
prior_2017 = train[train["date"]<=VAL_SPLIT].index
after_2017 = train[train["date"]>VAL_SPLIT].index

In [ ]:
train.index = train["date"]
train.drop("date",axis=1,inplace=True)

test.index = test["date"]
test.drop("date",axis=1,inplace=True)

# Split and Scale

In [ ]:
X = train.drop("num_sold", axis=1)
y= train["num_sold"]

In [ ]:
X_train = train.iloc[prior_2017,:].drop("num_sold", axis=1)
X_test = train.iloc[after_2017,:].drop("num_sold", axis=1)
y_train= train.iloc[prior_2017,:]["num_sold"]
y_test= train.iloc[after_2017,:]["num_sold"]

In [ ]:
X_train.head(2)

In [ ]:
# Removing  scaling with Catboost as there are categorical columns in training (and Im too lazy to scale only numerical and concat the categories)

X_train,X_test,test =scale_data(X_train,X_test,test)

# Optuna

In [ ]:
def objective(trial):
    # 2. Suggest values of the hyperparameters using a trial object.
    
    params = {
        "loss_function": trial.suggest_categorical("loss_function",  ["Poisson",'RMSE',"Tweedie"]),
        "eval_metric": trial.suggest_categorical("eval_metric",  ["Poisson",'RMSE',"Tweedie","SMAPE"]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 20),
        #"one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20),  
    }
    
    # Conditional Hyper-Parameters
    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1)
        
    #Tweedie needs variance power   
    if params["loss_function"] == "Tweedie":
        params["loss_function"] = "Tweedie:variance_power=" + str(trial.suggest_float("loss_function_tweedie", 1, 2))
    if params["eval_metric"] == "Tweedie":
        params["eval_metric"] = "Tweedie:variance_power=" + str(trial.suggest_float("eval_metric_tweedie", 1, 2))

    
    ## CREATE catboost model
    model = CatBoostRegressor(**params,
                              iterations=EPOCHS,
                              cat_features=categorical_feats
                         )
    model.fit(
        X_train, y_train, 
        eval_set=(X_test, y_test),
        early_stopping_rounds=EARLY_STOPPING,
        verbose=0,
        use_best_model=True,
        )
    
    #Predictions
    test_predictions = model.predict(X_test)
    smape = SMAPE(y_test,test_predictions)
    
    print("SMAPE:",smape)
    
    return smape

In [ ]:
if OPTUNA:
    print("RUNINING OPTUNA CATBOOST")
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=NUM_TRIALS)
    trial = study.best_trial

In [ ]:
if OPTUNA:
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial num :",trial.number)
    print(" SMAPE Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

# Run model

In [ ]:
if OPTUNA: 
    params = trial.params
    
else:
    #OPTUNA 300 - 7.604
    params = {
        "loss_function": "Poisson",
        "eval_metric": "Poisson",
        "learning_rate": 0.15643124768390415,
        "l2_leaf_reg": 0.034286809582276825,
        "depth": 4,
        "boosting_type": "Ordered",
        "bootstrap_type": "Bernoulli",
        "min_data_in_leaf": 9,
        "subsample": 0.5632840359532664
    }
        
        
    #Optuna 200
    """params = {
        #only if tweedie
        "loss_function":"RMSE",
        "eval_metric":"SMAPE",
        "learning_rate" : 0.08328012000638386,
        "l2_leaf_reg":0.011187384806520727,
        "depth": 10,
        "boosting_type": "Plain",
        'bootstrap_type': 'Bayesian',
        "min_data_in_leaf": 18,
        "bagging_temperature": 3.2583638997300923
        }"""

    #400 OPTUNA 
    """params = {
        #only if tweedie
        "loss_function":"Tweedie:variance_power=1.5811093354225167",
        "eval_metric":"RMSE",
        "learning_rate" : 0.5283210615574262,
        "l2_leaf_reg":0.9740971300008016,
        "depth": 5,
        'bootstrap_type': 'MVS',
        "boosting_type": "Ordered",
        "min_data_in_leaf": 15
        }"""

In [ ]:

def fit_model(X_train,y_train,X_test,y_test,test_df, seed_num):
    
    model = CatBoostRegressor(**params,
                              iterations=EPOCHS,
                          #cat_features=categorical_feats, 
                              random_seed=seed_num
                         )
    model.fit(
        X_train, y_train, 
        eval_set=(X_test, y_test),
        early_stopping_rounds=EARLY_STOPPING,
        verbose=0,
        use_best_model=True
        )


    train_preds = model.predict(X_test)
    #predict test data
    test_preds = model.predict(test_df)
    
    smape = SMAPE(y_test,train_preds)
    print("SMAPE:", smape)
    
    return train_preds, test_preds, smape

In [ ]:
train_preds, test_preds, smape = fit_model(X_train,y_train,X_test,y_test,test, 0)

## Random Seed CV
Variation of [adamwurdits](https://www.kaggle.com/adamwurdits/tps-01-2022-catboost-w-optuna-seed-averaging) 
I removed the CV as I wasnt having good scores with TimeSeriesSplit

In [ ]:
def seed_CV(X_train,y_train, X_test, y_test, test_df):
    
    smape_score = []
    seed_test_preds = np.zeros((SEED, len(test_df)))
    seed_train_preds = np.zeros((SEED, len(y_test)))
    
    for i in range(SEED):
        print(f"##### SEED {i} #######")
        
        train_preds, test_preds, smape = fit_model(X_train,y_train,X_test,y_test,test_df, i)

        smape_score.append(smape)
        seed_test_preds[i] = test_preds
        seed_train_preds[i] = train_preds

    
    return np.mean(smape_score) , np.mean(seed_test_preds,axis =0) , np.mean(seed_train_preds,axis =0)

final_smape, final_predictions, train_preds  = seed_CV(X_train,y_train, X_test, y_test, test)

In [ ]:
print("SMAPE :", final_smape)

In [ ]:
final_predictions

In [ ]:
print("SMAPE :",final_smape )
print(f"\n EPOCHS: {EPOCHS}")
print(f"\n SCALER: {SCALER_NAME}")
print(f"\n PARAMS: { params}")
print(f"\n Holidays : {HOLIDAYS}")
print(f"\n Next Holiday : {NEXT_HOLIDAY}")
print(f"\n Linear Date Augmentation : {LINEAR_DATE_AUG}")
print(f"\n POST_PROCESSING: {POST_PROCESSING}")

# Post Processing & Submission 

In [ ]:
sub = pd.read_csv("../input/tabular-playground-series-jan-2022/sample_submission.csv",index_col = 0)

In [ ]:
if POST_PROCESSING:
    # from previous run we are under predicting, lets scale the values upwards
    print("Scaling predictions ")
    print("preds_prior:", final_predictions)
    
    sub["num_sold"] = final_predictions*1.143
    
    print("preds after:", np.array(sub["num_sold"]))
else:
    sub["num_sold"] = final_predictions

In [ ]:
sub.to_csv("submission.csv")

In [ ]:
sub.head()

# Training Visualization

In [ ]:
#for visual only
test1 = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv",index_col = 0)
test1["date"] = pd.to_datetime(test1["date"])

fig,ax = plt.subplots(2,1, figsize=(25,20),sharey= True)

diff = y_test - train_preds
sns.lineplot(ax=ax[0], data= y_test, label="Train Actual",ci=None)
sns.lineplot(ax=ax[0], data = y_test,x = y_test.index , y = final_predictions, label ="Validation Prediction" ,ci=None)
sns.lineplot(ax=ax[0],data =sub, x= test1["date"], y = "num_sold",label="Final Prediction" ,ci=None) 

ax[0].set_title(f"Actual and Predicted Sales for {MODEL_TYPE}")

sns.lineplot(ax=ax[1], data = diff, label ="Residuals" )
ax[1].set_title(f"Residuals for {MODEL_TYPE} for 2018")

plt.show()

In [ ]:
plt.figure(figsize=(25,10))

sns.lineplot(data= train["num_sold"] ,label="Train Actual",ci=None)
sns.lineplot(data =sub, x= test1["date"], y = "num_sold",label="Final Prediction" ,ci=None) 
plt.title("Actual and Predicted Sales")

plt.show()

## Previous Rns 
#### Best Run
7.793541049903422  loss_function': 'RMSE', 'eval_metric': 'SMAPE', 'learning_rate': 0.08328012000638386, 'l2_leaf_reg': 0.011187384806520727, 'depth': 10, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'min_data_in_leaf': 18, 'bagging_temperature': 3.2583638997300923

####Best Summited
7.993  ---Optuna 200   - --Submitted as 5.75958    ---no holidays, 

def engineer(df):
    #get GDP from file 
    df["gdp"] = df.apply(get_gdp, axis=1)   #improves Huber & Tweedie & catboost
    
    df["day"] = df["date"].dt.day
    df["dayofweek"] = df["date"].dt.dayofweek
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    
    #play around with if Tree model - each varies 
    df['dayofyear'] = df['date'].dt.dayofyear                ### This can cause noise 
    #df['inverse_dayofyear'] = 365 - df['date'].dt.dayofyear    # good for all  except catboost
    #df.loc[df["year"] == 2016 , "inverse_dayofyear"] = df.loc[df["year"] == 2016 , "inverse_dayofyear"]+1     #Leap year in 2016
    #df['quarter'] = 'Q' + df['date'].dt.quarter.astype(str)      # Good for lightgbm & Huber, bad for Tweedie & catboost
    #df['daysinmonth'] = df['date'].dt.days_in_month           ## Also reduces performance
     
    #encoding
    for country in ['Finland', 'Norway']:
        df[country] = df.country == country
    df['KaggleRama'] = df.store == 'KaggleRama'
    for product in ['Kaggle Mug', 'Kaggle Sticker']:
        df[product] = df['product'] == product
    
    #df["Friday"] = df["dayofweek"] ==4
    #df["Sat_sun"] = (df["dayofweek"] ==5) |(df["dayofweek"] ==6)
    
    df.drop(["country","store","product"], axis =1, inplace = True)
    
    # Seasonal variations (Fourier series)
    # The three products have different seasonal patterns
    dayofyear = df.date.dt.dayofyear
    for k in range(1, 20):
        df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
        df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
        df[f'mug_sin{k}'] = df[f'sin{k}'] * df['Kaggle Mug']
        df[f'mug_cos{k}'] = df[f'cos{k}'] * df['Kaggle Mug']
        df[f'sticker_sin{k}'] = df[f'sin{k}'] * df['Kaggle Sticker']
        df[f'sticker_cos{k}'] = df[f'cos{k}'] * df['Kaggle Sticker']

    return df